In [0]:
# Specify root path to process data 
dbutils.widgets.text('root_path', 's3a://strava-data-lake/')
root_path = dbutils.widgets.get('root_path')
delta_bronze_path = f"{root_path}bronze-tables/"
delta_silver_path = f"{root_path}silver-tables/"
delta_gold_path = f"{root_path}gold-tables/"
incoming_path = f"{root_path}incoming-files/"
print(f"Delta Bronze Path: {delta_bronze_path}\n Incoming File Path: {incoming_path}")

Delta Bronze Path: s3a://strava-data-lake/bronze-tables/
 Incoming File Path: s3a://strava-data-lake/incoming-files/


In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window

In [0]:
spark.sql(f"CREATE TABLE IF NOT EXISTS default.stg_strava_ingestion LOCATION '{delta_bronze_path}'")

Out[7]: DataFrame[]

In [0]:
spark.sql(f"COPY INTO default.stg_strava_ingestion \
FROM '{incoming_path}' \
FILEFORMAT = CSV \
FORMAT_OPTIONS ('header' = 'true') \
COPY_OPTIONS('mergeSchema' ='true')")

Out[9]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint, num_skipped_corrupt_files: int]

In [0]:
%sql
-- Creating the silver table and schema so our data load can be used by gold level tables for analytics ;
CREATE TABLE IF NOT EXISTS default.stg_strava_silver(
activity_id STRING NOT NULL,
activity_name STRING,
activity_start_date TIMESTAMP,
activity_start_date_local TIMESTAMP,
activity_timezone STRING,
activity_distance FLOAT,
activity_moving_time INT,
activity_elapsed_time INT,
activity_total_elevation_gain FLOAT,
activity_type STRING,
activity_workout_type STRING,
activity_location_country STRING,
activity_achievement_count INT,
activity_kudos_count INT,
activity_comment_count INT,
activity_athlete_count INT,
activity_average_speed FLOAT,
activity_max_speed FLOAT,
activity_average_cadence FLOAT,
activity_average_temp FLOAT,
activity_average_heartrate FLOAT,
activity_max_heartrate FLOAT,
acvitity_suffer_score INT,
activity_lat FLOAT,
acvitity_lng FLOAT
)
USING DELTA 
LOCATION 's3a://strava-data-lake/silver-tables/';

In [0]:
%sql
INSERT INTO default.stg_strava_silver SELECT * FROM default.stg_strava_ingestion

num_affected_rows,num_inserted_rows
8,8


In [0]:
display(spark.sql(f"SELECT COUNT(activity_id) AS Activities,
SELECT CAST(MONTH(activity_start_date_local) AS VARCHAR) + '/' + CAST(YEAR(activity_start_date_local) AS VARCHAR)' AS Activity_Month \
FROM stg_strava_silver \
GROUP BY MONTH(activity_start_date_local) \
ORDER BY MONTH(activity_start_date_local)"))

count(activity_id),Activity_Month
24,5
16,6
8,7
